In [ ]:
from IPython.display import display
import re
from cameo import models
from cameo.strain_design import pathway_prediction
from os.path import dirname, join, pardir
from cobra.io import read_sbml_model
from dfba import DfbaModel, ExchangeFlux, KineticVariable

ModuleNotFoundError: No module named 'dfba'

In [ ]:
pip install dfba


  Using cached dfba-0.1.8.tar.gz (63 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 0.0/16.7 MB ? eta -:--:--
    --------------------------------------- 0.4/16.7 MB 12.9 MB/s eta 0:00:02
   - -------------------------------------- 0.8/16.7 MB 9.8 MB/s eta 0:00:02
   -- ------------------------------------- 1.0/16.7 MB 11.1 MB/s eta 0:00:02
   --- ------------------------------------ 1.5/16.7 MB 8.8 MB/s eta 0:00:02
   ---- ----------------------------------- 1.8/16.7 MB 8.9 MB/s eta 0:00:02
   ----- ---------------------------------- 2.1/16.7 MB 8.9 MB/s eta 0:00:02
   ----- ---------------------------------- 2.1/16.7 MB 8.9 MB/s eta 0:00:02
   ----- ---------

  error: subprocess-exited-with-error
  
  × Building wheel for dfba (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [40 lines of output]
      C:\Users\pauli\AppData\Local\Temp\pip-build-env-j2ma4yg7\overlay\Lib\site-packages\setuptools\config\setupcfg.py:293: _DeprecatedConfig: Deprecated config in `setup.cfg`
      !!
      
              ********************************************************************************
              The license_file parameter is deprecated, use license_files instead.
      
              This deprecation is overdue, please update your project and remove deprecated
              calls to avoid build errors in the future.
      
              See https://setuptools.pypa.io/en/latest/userguide/declarative_config.html for details.
              ********************************************************************************
      
      !!
        parsed = self.parsers.get(option_name, lambda x: x)(value)
      running bdist_wheel
     

In [ ]:
conda install -c conda-forge dfba

Solving environment: ...working... unsuccessful initial attempt using frozen solve. Retrying with flexible solve.
Solving environment: ...working... unsuccessful initial attempt using frozen solve. Retrying with flexible solve.

Note: you may need to restart the kernel to use updated packages.


DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): repo.anaconda.com:443
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): repo.anaconda.com:443
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): repo.anaconda.com:443
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): repo.anaconda.com:443
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): repo.anaconda.com:443
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): repo.anaconda.com:443
DEBUG:urllib3.connectionpool:https://repo.anaconda.com:443 "GET /pkgs/r/win-64/current_repodata.json HTTP/1.1" 304 0
DEBUG:urllib3.connectionpool:https://repo.anaconda.com:443 "GET /pkgs/main/noarch/current_repodata.json HTTP/1.1" 304 0
DEBUG:urllib3.connectionpool:https://repo.anaconda.com:443 "GET /pkgs/main/win-64/current_repodata.json HTTP/1.1" 304 0
DEBUG:urllib3.connectionpool:https://repo.anaconda.com:443 "GET /pkgs/r/noarch/current_repodata.json HTTP/1.1" 304 0
DEBUG:urll

In [ ]:
pip install --upgrade wheel

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import dfba


In [ ]:
conda install -c conda-forge dfba

In [ ]:
import dfba

: 

In [ ]:
conda install -p c:\Users\pauli\OneDrive\Dokumenty\DTU\Cell factories\group-assingment-team11\.conda ipykernel --update-deps --force-reinstall

: 